pre-trained eng_flair_nner_distilbert.pt

demo: https://corpling.uis.georgetown.edu/xrenner/

Model: https://corpling.uis.georgetown.edu/amir/download/

Code: https://github.com/amir-zeldes/xrenner

Doc: https://corpling.uis.georgetown.edu/xrenner/doc/

In [1]:
# pre-trained eng_flair_nner_distilbert.pt
# Demo: https://corpling.uis.georgetown.edu/xrenner/
# Model: https://corpling.uis.georgetown.edu/amir/download/
# Code: https://github.com/amir-zeldes/xrenner
# Doc: https://corpling.uis.georgetown.edu/xrenner/doc/

import json
import random

import stanza
from stanza.utils.conll import CoNLL

from xrenner import Xrenner

stanza.download('en')
nlp_stanza = stanza.Pipeline('en')

2021-06-24 00:30:16 INFO: Downloading default packages for language: en (English)...
2021-06-24 00:30:18 INFO: File exists: /home/he/stanza_resources/en/default.zip.
2021-06-24 00:30:28 INFO: Finished downloading models and saved to /home/he/stanza_resources.
2021-06-24 00:30:28 INFO: Loading these models for language: en (English):
| Processor | Package   |
-------------------------
| tokenize  | combined  |
| pos       | combined  |
| lemma     | combined  |
| depparse  | combined  |
| sentiment | sstplus   |
| ner       | ontonotes |

2021-06-24 00:30:28 INFO: Use device: gpu
2021-06-24 00:30:28 INFO: Loading: tokenize
2021-06-24 00:30:30 INFO: Loading: pos
2021-06-24 00:30:30 INFO: Loading: lemma
2021-06-24 00:30:30 INFO: Loading: depparse
2021-06-24 00:30:31 INFO: Loading: sentiment
2021-06-24 00:30:32 INFO: Loading: ner
2021-06-24 00:30:32 INFO: Done loading processors!


In [2]:
with open("./data/subtask3-coreference/en-train.json") as fp:
    entries = fp.readlines()

In [3]:
data = []
for entry in entries:
    data.append(json.loads(entry))

In [4]:
print("Examples in the dataset:")
random.sample(data, 2)

Examples in the dataset:


[{'event_clusters': [[6], [1]],
  'sentence_no': [1, 6],
  'sentences': ['Several hundred people who have occupied homes at Delft on the Cape Flats are on their way to the Cape High Court in a bid to block their impending eviction , a spokesman said .',
   'Speaking just after 1pm , Poni said he and the hundreds of residents who had demonstrated outside the court were now on their way by train to the High Court , where they would make another attempt to get an urgent interdict .'],
  'id': 55550},
 {'event_clusters': [[3, 4], [1, 7]],
  'sentence_no': [1, 3, 4, 7],
  'sentences': ['Flash strike by guards delays suburban trains - Indian Express Express News Service , Express News Service : Fri Nov 30 2012 , 03:16 hrs Most guards on local trains abstained from work Thursday protesting " irritable " behaviour of the area officer at Churchgate , forcing station masters and traffic inspectors to take up their role .',
   'Four days ago , guard Rakesh Jha had gone on a hunger strike at Churc

In [5]:
print("Number of entries")
len(data)

Number of entries


596

In [6]:
sentence_no_list = []
event_clusters_list = []

for d in data:
    sentence_no_list += d['sentence_no']
    event_clusters_list += d['event_clusters']

In [7]:
# Label data
for data_entry in data:
    _id = data_entry['id']
    _docu = data_entry['sentences']
    _docu_in = [stanza.Document([],text=d) for d in _docu]
    _docu_out = nlp_stanza(_docu_in)
    data_entry['stanza'] = _docu_out

In [18]:
# Convert Stanza results to CoNLL10 format
for data_entry in data:
    data_entry['conll'] = [CoNLL.convert_dict(s.to_dict()) for s in data_entry['stanza']]
    for ec in data_entry['conll']:
        _ec = ec[0]
        _tmp = []
        for _ecc in _ec:
            _tmp.append('\t'.join(_ecc))
        data_entry['conll_text'] = '\n'.join(_tmp)+'\n\n'

In [29]:
nlp_stanza(data[0]['sentences'][0]).entities

[{
   "text": "Pakistan",
   "type": "GPE",
   "start_char": 10,
   "end_char": 18
 },
 {
   "text": "along border October 12",
   "type": "EVENT",
   "start_char": 54,
   "end_char": 77
 },
 {
   "text": "2014",
   "type": "DATE",
   "start_char": 80,
   "end_char": 84
 },
 {
   "text": "Parishad",
   "type": "GPE",
   "start_char": 187,
   "end_char": 195
 },
 {
   "text": "Saturday",
   "type": "DATE",
   "start_char": 208,
   "end_char": 216
 },
 {
   "text": "Pakistan",
   "type": "GPE",
   "start_char": 280,
   "end_char": 288
 }]

In [32]:
nlp_stanza(data[0]['sentences'][1]).entities


[{
   "text": "Union",
   "type": "ORG",
   "start_char": 41,
   "end_char": 46
 }]

In [9]:
xrenner = Xrenner()

/home/he/anaconda3/envs/xrenner/lib/python3.6/site-packages/sklearn/base.py:315: UserWarning: Trying to unpickle estimator LabelEncoder from version 0.22.1 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/home/he/anaconda3/envs/xrenner/lib/python3.6/site-packages/sklearn/base.py:315: UserWarning: Trying to unpickle estimator OrdinalEncoder from version 0.22.1 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [27]:
xrenner.analyze(data[0]['conll_text'], out_format='conll')

AttributeError: 'BasicTokenizer' object has no attribute 'strip_accents'